In [1]:
import pandas as pd
import numpy as np

from rl_lib.swiss_round.environment import SwissRoundEnv
%reload_ext autoreload
%autoreload 2

In [ ]:
n_teams = 18
n_rounds = 6
team_strengths = [1 - 0.05 * i for i in range(n_teams)]
threshold_ranks = [4,12]
bonus_points = [20,10]

env = SwissRoundEnv(
    n_teams=n_teams,
    n_rounds=n_rounds,
    team_strengths=team_strengths,
    threshold_ranks=threshold_ranks,
    bonus_points=bonus_points,
    agent_id=None,
    max_draw_probability=0.3
)

--- Simulating round n°1 ---
Game : Team 5 (points : 0, strength : 0.75) vs Team 12 (points : 0, strength : 0.40) : Team 5 wins
Game : Team 8 (points : 0, strength : 0.60) vs Team 17 (points : 0, strength : 0.15) : Draw
Game : Team 2 (points : 0, strength : 0.90) vs Team 10 (points : 0, strength : 0.50) : Team 2 wins
Game : Team 15 (points : 0, strength : 0.25) vs Team 1 (points : 0, strength : 0.95) : Team 1 wins
Game : Team 11 (points : 0, strength : 0.45) vs Team 13 (points : 0, strength : 0.35) : Team 13 wins
Game : Team 3 (points : 0, strength : 0.85) vs Team 16 (points : 0, strength : 0.20) : Draw
Game : Team 7 (points : 0, strength : 0.65) vs Team 9 (points : 0, strength : 0.55) : Team 9 wins
Game : Team 14 (points : 0, strength : 0.30) vs Team 6 (points : 0, strength : 0.70) : Team 14 wins
Game : Team 0 (points : 0, strength : 1.00) vs Team 4 (points : 0, strength : 0.80) : Draw
--- Simulating round n°2 ---
Game : Team 1 (points : 3, strength : 0.95) vs Team 2 (points : 3, stre

In [4]:
simulation = env.simulate_n_tournaments(1000,24, display_results=True)

Simulating tournaments: 100%|██████████| 1000/1000 [00:06<00:00, 160.45it/s]


Simulation Results (from 1000 tournaments using 24 cores):
Team | Strength | Avg Points | Avg Rank | Top-4 % | Top-12 % | 
------------------------------------------------------
   0 |     1.00 |      10.24 |     6.83 |  40.83% |  83.99% | 
   1 |     0.95 |       9.68 |     7.59 |  32.62% |  79.64% | 
   2 |     0.90 |       9.66 |     7.65 |  34.25% |  79.43% | 
   3 |     0.85 |       9.39 |     8.02 |  31.10% |  78.12% | 
   4 |     0.80 |       9.02 |     8.62 |  26.14% |  73.86% | 
   5 |     0.75 |       8.87 |     8.80 |  26.65% |  71.73% | 
   6 |     0.70 |       9.01 |     8.63 |  25.23% |  74.87% | 
   7 |     0.65 |       8.58 |     9.32 |  22.90% |  69.10% | 
   8 |     0.60 |       8.41 |     9.58 |  20.97% |  66.16% | 
   9 |     0.55 |       8.50 |     9.42 |  22.19% |  68.09% | 
  10 |     0.50 |       8.18 |     9.89 |  18.74% |  65.45% | 
  11 |     0.45 |       7.96 |    10.25 |  17.33% |  62.61% | 
  12 |     0.40 |       7.95 |    10.26 |  16.92% |  61.40% | 
  

In [ ]:
# Simulate tournament
final_standings = env.simulate_tournament(verbose= True)

print("\nFinal standings (team_id, points, opponent_average):")
for rank, (team_id, points, opp_avg,strength) in enumerate(final_standings, 1):

    print(f"Rank {rank}: Team {team_id} - Strength {strength:.2f} - Points: {points} - Opponent Avg: {opp_avg:.2f}")
